In [11]:
from keras.callbacks import TensorBoard
from keras.utils.vis_utils import plot_model

In [7]:
import pandas as pd
import numpy as np
import scipy
import sys
import imp

In [8]:
from process_data import load_data,get_rank_of_size
from data_util import unpack_news_data,data_generator
import models
import callbacks
imp.reload(models)
imp.reload(callbacks)

<module 'callbacks' from '/home/beginner/code/stock/callbacks.py'>

In [4]:
rank = get_rank_of_size()
rank.remove('XOM')
rank.remove('GOOG')

In [5]:
data = load_data(rank[:1],encoder_kind='Bert')

1675 1156 1168
116 62 74
108 41 53
data of FB is loaded


In [ ]:
dd = data_generator(b_size,(x1_train,x2_train,y_train),unpack_news_data)

In [ ]:
print(rank)

In [ ]:
# sparse matrix
data[rank[0]]['train'][0][0][0].toarray().shape

# TEXTUAL MODEL

In [9]:
model = models.build_textual_model()

Instructions for updating:
Colocations handled automatically by placer.


In [13]:
print(model.summary())
plot_model(model, to_file='tmodel.png',show_shapes=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 7, 50, 768)        0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 7, 50, 768)        0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 7, 768)            7700      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 7, 100)            76900     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 7, 100)            45300     
_________________________________________________________________
attention_layer_2 (Attention (None, 100)               1020      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
__________

In [ ]:
b_size = 64
x_train = np.concatenate([data[key]['train'][0] for key in data],axis=0)
y_train = np.concatenate([data[key]['train'][2] for key in data],axis=0)
x_val = unpack_news_data(np.concatenate([data[key]['val'][0] for key in data],axis=0))
y_val = np.concatenate([data[key]['val'][2] for key in data],axis=0)
cbs = [TensorBoard(log_dir='model_log/textual/')]
steps = (len(x_train)+b_size-1)//b_size

In [ ]:
model.fit_generator(generator=data_generator(b_size,(x_train,y_train),unpack_news_data),
                    samples_per_epoch=steps,
                    validation_data=(x_val,y_val),
                    callbacks = cbs,
                    epochs=20,
                    verbose=1,
                   )

# NUMERICAL MODEL

In [20]:
nmodel = models.build_numerical_model()

In [21]:
print(nmodel.summary())
plot_model(nmodel, to_file='nmodel.png',show_shapes=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20, 5)             0         
_________________________________________________________________
gru_1 (GRU)                  (None, 20, 100)           31800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 100)           0         
_________________________________________________________________
gru_2 (GRU)                  (None, 100)               60300     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 22        
Total para

In [ ]:
b_size = 64
x_train = np.concatenate([data[key]['train'][1] for key in data],axis=0)
y_train = np.concatenate([data[key]['train'][2] for key in data],axis=0)
x_val = np.concatenate([data[key]['val'][1] for key in data],axis=0)
y_val = np.concatenate([data[key]['val'][2] for key in data],axis=0)
cbs = [TensorBoard(log_dir='model_log/numerical/')]
steps = (len(x_train)+b_size-1)//b_size

In [ ]:
nmodel.fit_generator(generator=data_generator(b_size,(x_train,y_train)),
                    samples_per_epoch=steps,
                    validation_data=(x_val,y_val),
                    callbacks = cbs,
                    epochs = 20,
                    verbose=1,
                   )

# HYBRID MODEL

In [31]:
hmodel = models.build_hybrid_model()

In [32]:
print(hmodel.summary())
plot_model(hmodel, to_file='hmodel.png',show_shapes=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 7, 50, 768)   0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 7, 50, 768)   0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 20, 5)        0                                            
__________________________________________________________________________________________________
time_distributed_2 (TimeDistrib (None, 7, 768)       7700        time_distributed_1[0][0]         
__________________________________________________________________________________________________
gru_2 (GRU

In [ ]:
b_size = 64
x1_train = np.concatenate([data[key]['train'][0] for key in data],axis=0)
x2_train = np.concatenate([data[key]['train'][1] for key in data],axis=0)
x1_val = unpack_news_data(np.concatenate([data[key]['val'][0] for key in data],axis=0))
x2_val = np.concatenate([data[key]['val'][1] for key in data],axis=0)
y_val = np.concatenate([data[key]['val'][2] for key in data],axis=0)
y_train = np.concatenate([data[key]['train'][2] for key in data],axis=0)
cbs = [TensorBoard(log_dir='num_log/hybrid/')]
steps = (len(x1_train)+b_size-1)//b_size

In [ ]:
hmodel.fit_generator(generator=data_generator(b_size,(x1_train,x2_train,y_train),unpack_news_data),
                    samples_per_epoch=steps,
                    validation_data=([x1_val,x2_val],y_val),
                    callbacks = cbs,
                    epochs=10,
                    verbose=1,
                   )

# Random Guess

In [23]:
gmodel = models.build_guess_model()

In [24]:
x_train = np.concatenate([np.random.randint(2, size=(data[key]['val'][2].shape[0],1)) for key in data])
y_train = np.concatenate([data[key]['val'][2] for key in data])

In [ ]:
gmodel.fit(x=x_train,y=y_train,verbose=1,epochs=30,batch_size=32)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
random_guess_1 (RandomGuess) (None, 2)                 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
result_dict = {}
for code in rank[:20]:
    result = hmodel.evaluate(x=[unpack_news_data(data[code]['test'][0]),data[code]['test'][1]],
                             y=data[code]['test'][2],batch_size=32)
    result_dict[code] = result[1]
    if 'avg' not in result_dict:
        result_dict['avg'] = 0
    result_dict['avg'] += result[1]
result_dict['avg'] /= 20
print(result_dict)

In [33]:
from keras import backend as K
K.clear_session()